In [30]:
import nltk
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [31]:
fname_all_anecdotal_sents = "/home/shirish/BTECHSEM2/project/books/stories/new_complete_stories.txt"
fname_all_non_anecdotal_sents = "/home/shirish/BTECHSEM2/project/books/annotated_books/stories_without_anecdotes/new_all_stories.txt2"
#fname_all_stories_sents = "/home/shirish/BTECHSEM2/project/books/combined_book/all_stories.txt"
#fname_buffet_sents = "/home/shirish/BTECHSEM2/project/books/buffet_en.txt"
#imp_cols = ["NNP","VB", "VBD", "VBG"]#, "VBN", "VBP", "VBZ"]
imp_cols = ["POS", "NNP", "VBD", "VBG", "PRP", "VBZ"]

In [32]:
#fname_all_anecdotal_sents = "/home/shirish/BTECHSEM2/project/Book/anecdotal_sents/all_anecdotes.txt"
#fname_all_non_anecdotal_sents = "/home/shirish/BTECHSEM2/project/Book/non_anecdotal_sents/all_non_anecdotal.txt"
#fname_all_stories_sents = "/home/shirish/BTECHSEM2/project/books/combined_book/all_stories.txt"
#fname_buffet_sents = "/home/shirish/BTECHSEM2/project/books/buffet_en.txt"
#imp_cols = ["NNP","VB", "VBD", "VBG"]#, "VBN", "VBP", "VBZ"]

In [33]:
def tokenize(fname):
    """
    @param fname = filename
    Returns = list of sentences, where each sentence is a list of POS tagged words
    """
    f = open(fname, "r")
    taggedSents = []
    s = f.read()
    s = unicode(s, errors="ignore")
    sentences = nltk.sent_tokenize(s)
    for i in sentences:
        taggedSents.append(nltk.pos_tag(nltk.word_tokenize(i)))
    return taggedSents

In [34]:
def tagSentences():
    """
    This function returns two tagged sentences list
    ie List of non_anecdotal , list of anecdotal sentences
    """
    taggedSents1 = tokenize(fname_all_anecdotal_sents)
    taggedSents0 = tokenize(fname_all_non_anecdotal_sents)
    return taggedSents0, taggedSents1

In [35]:
def featureset_df(taggedSents, value, imp_cols):
    """
    This function returns a dataframe consisting of the imp_cols and target column after
    removing the NaN values
    @param taggedSents = list of sentences, where each sentence is a list of POS tagged words
    @param value = Value given to the target column in the dataframe
    Returns = dataframe with columns as imp_cols and target with value @param value
    """
    sents1 = []
    for i in taggedSents:
        l = {}
        for j in i:
            if j[1].isalpha():
                # If Noun
                if j[1][0] == "NNP":# or j[1] == "PRP":
                    l["NNP"] = 1
                else:
                    l[j[1]] = 1
        sents1.append(l)
    df = pd.DataFrame(sents1)
    df.fillna(0, inplace=True)
    features = df[imp_cols]
    features["target"] = value
    return features

In [36]:
def create_df_of_sentences():
    global sents1, sents0
    f1 = open(fname_all_anecdotal_sents)
    f0 = open(fname_all_non_anecdotal_sents)
    s1 = f1.read()
    s0 = f0.read()
    s1 = unicode(s1, errors = "ignore")
    s0 = unicode(s0, errors= "ignore")
    sents1 =nltk.sent_tokenize(s1)
    sents0 = nltk.sent_tokenize(s0)
    

In [37]:
def create_feature_set_df(taggedSents0, taggedSents1):
    """
    Call this function after calling tagSentences()
    This function returns the features dataframe
    """
    features0 = featureset_df(taggedSents0, 0, imp_cols)
    if(taggedSents1 is not None):
        features1 = featureset_df(taggedSents1, 1, imp_cols)
        features = pd.concat([features1, features0])
    else:
        features = features0
    return features
    

In [38]:
def init():
    TS0, TS1 = tagSentences()
    df = create_feature_set_df(TS0, TS1)
    init2(TS0, TS1, df)

In [39]:
def init2(TS0, TS1, df):   
    df = shuffle(df)
    X_train, X_test, y_train, y_test = train_test_split(df[imp_cols], df["target"], test_size=0.25)    
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.5))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    print(model.summary())
    Y = pd.get_dummies(data['Target']).values
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10, random_state = 42)
    print(X_train.shape,Y_train.shape)
    print(X_test.shape,Y_test.shape)
    batch_size = 32
    model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size)    
    score, acc =  model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("score: %.2f" % (score))
    print("acc: %.2f" % (acc))
    
    #from matplotlib import pyplot
    #history = model.fit(X, Y, epochs=10, validation_data=(X_test, Y_test))
    #pyplot.plot(history.history['loss'])
    #pyplot.plot(history.history['val_loss'])
    #pyplot.title('model train vs validation loss')
    #pyplot.ylabel('loss')
    #pyplot.xlabel('epoch')
    #pyplot.legend(['train', 'validation'], loc='upper right')
    #pyplot.show()


In [66]:
init()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 75, 128)           256000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 75, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None
((1285, 75), (1285, 2))
((143, 75), (143, 2))
Epoch 1/7
1285/1285 [==============================] - 13s 10ms/step - loss: 0.6788 - acc: 0.5743
Epoch 2/7
1285/1285 [==============================] - 13s 10ms/step - loss: 0.6415 - acc: 0.6389
Epoch 3/7
1285/1285 [

In [53]:
create_df_of_sentences()

In [54]:
d = [{'Text': i, 'Target': 'Anecdotal'} for i in sents1]

In [55]:
d.extend([{'Text': i, 'Target': 'NonAnecdotal'} for i in sents0])

In [56]:
data = pd.DataFrame(d)


In [57]:
data.Target.value_counts()

NonAnecdotal    825
Anecdotal       603
Name: Target, dtype: int64

In [58]:
data.head(1)

,Target,Text
0,Anecdotal,The power of embracing failure is illustrated ...


In [59]:
data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [60]:
data["Target"].value_counts()

NonAnecdotal    825
Anecdotal       603
Name: Target, dtype: int64

In [61]:
max_features = 2000 # 95%acc
#max_features = 1000

In [62]:
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)

In [63]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
#print(model.summary())

In [64]:
Y = pd.get_dummies(data['Target']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

((1071, 75), (1071, 2))
((357, 75), (357, 2))


In [231]:
batch_size = 32
history = model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, validation_data=(X_test, Y_test))

Train on 1071 samples, validate on 357 samples
Epoch 1/3
1071/1071 [==============================] - 14s 13ms/step - loss: 0.6842 - acc: 0.5696 - val_loss: 0.6680 - val_acc: 0.6022
Epoch 2/3
1071/1071 [==============================] - 10s 9ms/step - loss: 0.6284 - acc: 0.6424 - val_loss: 0.6028 - val_acc: 0.6639
Epoch 3/3
1071/1071 [==============================] - 10s 10ms/step - loss: 0.5126 - acc: 0.7479 - val_loss: 0.6173 - val_acc: 0.6611


In [233]:

score, acc =  model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.62
acc: 0.66


In [223]:
model.predict_classes(X_test)

array([0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,

In [235]:
from matplotlib import pyplot
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X, Y, epochs=10, validation_data=(X_test, Y_test))
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()


Train on 1428 samples, validate on 357 samples
Epoch 1/10
1428/1428 [==============================] - 18s 12ms/step - loss: 0.6751 - acc: 0.5840 - val_loss: 0.6582 - val_acc: 0.6751
Epoch 2/10
1428/1428 [==============================] - 13s 9ms/step - loss: 0.6152 - acc: 0.6786 - val_loss: 0.5547 - val_acc: 0.7591
Epoch 3/10
 128/1428 [=>............................] - ETA: 10s - loss: 0.5227 - acc: 0.8125

KeyboardInterrupt: 